In [1]:
import string
import pickle
from keras.models import load_model
from nltk.tokenize import word_tokenize
import re
from nltk.corpus import stopwords
stop=set(stopwords.words('english'))
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

In [2]:
def receive_text():
    entered_text = input("Please enter a string (Press Enter to finish): ")
    if not entered_text:
        return
    return entered_text

In [4]:
entered_text = receive_text()

Please enter a string (Press Enter to finish): Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all


In [5]:
def data_cleaning(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    text = url.sub(r'', text)
    
    html = re.compile(r'<.*?>')
    text = html.sub(r'', text)
    
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
                               
    table=str.maketrans('','',string.punctuation)
    text = text.translate(table)
    
    text = text.lower()
    return text
                               

In [6]:
entered_text = data_cleaning(entered_text)

In [7]:
print(entered_text)

our deeds are the reason of this earthquake may allah forgive us all


In [8]:
def create_corpus(text):
    corpus = []
    words = [word.lower() for word in word_tokenize(text) if((word.isalpha()==1) & (word not in stop))]
    corpus.append(words)
    return corpus

In [9]:
entered_text = create_corpus(entered_text)

In [10]:
print(entered_text)

[['deeds', 'reason', 'earthquake', 'may', 'allah', 'forgive', 'us']]


In [11]:
with open('./model/tokenize_model/tokenizer_obj.pkl', 'rb') as file:
    tokenizer_obj = pickle.load(file)

In [14]:
MAX_LEN = 50
sequences=tokenizer_obj.texts_to_sequences(entered_text)
tweet_pad=pad_sequences(sequences,maxlen=MAX_LEN,truncating='post',padding='post')

In [15]:
print(tweet_pad)

[[5467  738  175   80 1805 3529   16    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0]]


In [16]:
model = load_model('./model/studyed_model/desaster.h5')

In [17]:
predict = model.predict(tweet_pad)

1/1 [==============================] - 4s 4s/step


In [18]:
print(predict)

[[0.47774145]]


In [19]:
if predict >= 0.5:
    print("disaster")
else:
    print("not disaster")

not disaster


In [55]:

# 1. Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all  :disaster
# 2. 13,000 people receive #wildfires evacuation orders in California  :disaster
# 3. @bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C  :disaster
# 4. My car is so fast  :not
# 5. What a goooooooaaaaaal!!!!!!  :not
# 6. #AFRICANBAZE: Breaking news:Nigeria flag set ablaze in Aba. http://t.co/2nndBGwyEi  :disaster 